In [67]:
import pandas as pd
import json
import numpy as np
from datetime import datetime
import math
import time
from datetime import datetime

In [ ]:
df = pd.read_csv('train_v2.csv')

In [ ]:
#Label json columns
json_columns = ['device', 'geoNetwork', 'totals', 'trafficSource']

In [ ]:
# Expand json columns
for cols in json_columns:
    df_norm = pd.json_normalize([json.loads(val) for val in df[cols]]).fillna(0)
    df = pd.merge(df, df_norm, left_index=True, right_index=True)

In [ ]:
df.shape

In [ ]:
# Randomly take a fraction of the data
df_2 = df.sample(frac = 0.05)

In [ ]:
df_2.to_csv('df_2.csv', index=False)

In [68]:
df_2 = pd.read_csv('df_2.csv')

/Users/kasey.larsen/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [69]:
pd.set_option('display.max_rows', df_2.shape[0]+1)

In [70]:
df_2.shape

(85417, 64)

In [71]:
df_2.head()

,channelGrouping,customDimensions,date,device,fullVisitorId,geoNetwork,hits_x,socialEngagementType,totals,trafficSource,...,adwordsClickInfo.criteriaParameters,referralPath,isTrueDirect,adContent,adwordsClickInfo.page,adwordsClickInfo.slot,adwordsClickInfo.gclId,adwordsClickInfo.adNetworkType,adwordsClickInfo.isVideoAd,campaignCode
0,Social,"[{'index': '4', 'value': 'South America'}]",20161026,"{""browser"": ""Safari"", ""browserVersion"": ""not a...",6650845619874507930,"{""continent"": ""Americas"", ""subContinent"": ""Sou...","[{'hitNumber': '1', 'time': '0', 'hour': '4', ...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""referralPath"": ""/yt/about/pt-BR/"", ""campaign...",...,not available in demo dataset,/yt/about/pt-BR/,0,0,0,0,0,0,0,0
1,Organic Search,[],20171125,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",2340120593107979567,"{""continent"": ""Asia"", ""subContinent"": ""Souther...","[{'hitNumber': '1', 'time': '0', 'hour': '7', ...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",...,not available in demo dataset,0,0,0,0,0,0,0,0,0
2,Organic Search,"[{'index': '4', 'value': 'North America'}]",20170915,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",5862581318107779527,"{""continent"": ""Americas"", ""subContinent"": ""Nor...","[{'hitNumber': '1', 'time': '0', 'hour': '8', ...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""1"", ""pageviews"": ""1"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",...,not available in demo dataset,0,0,0,0,0,0,0,0,0
3,Organic Search,[],20180324,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",3865236667712428359,"{""continent"": ""Asia"", ""subContinent"": ""Southea...","[{'hitNumber': '1', 'time': '0', 'hour': '23',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""3"", ""pageviews"": ""3"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",...,not available in demo dataset,0,0,0,0,0,0,0,0,0
4,Organic Search,[],20170625,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",5807128789237686268,"{""continent"": ""Asia"", ""subContinent"": ""Souther...","[{'hitNumber': '1', 'time': '0', 'hour': '7', ...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""3"", ""pageviews"": ""3"",...","{""campaign"": ""(not set)"", ""source"": ""(direct)""...",...,not available in demo dataset,0,0,0,0,0,0,0,0,0


In [72]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85417 entries, 0 to 85416
Data columns (total 64 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   channelGrouping                      85417 non-null  object
 1   customDimensions                     85417 non-null  object
 2   date                                 85417 non-null  int64 
 3   device                               85417 non-null  object
 4   fullVisitorId                        85417 non-null  object
 5   geoNetwork                           85417 non-null  object
 6   hits_x                               85417 non-null  object
 7   socialEngagementType                 85417 non-null  object
 8   totals                               85417 non-null  object
 9   trafficSource                        85417 non-null  object
 10  visitId                              85417 non-null  int64 
 11  visitNumber                          8541

In [73]:
#Drop columns missing larger numbers of data or are closely linked to other columns
df_2 = df_2.drop([ 'browserVersion', 'browserSize', 'operatingSystemVersion',
                  'mobileDeviceBranding', 'adwordsClickInfo.criteriaParameters', 'mobileDeviceModel', 
                  'mobileInputSelector', 'mobileDeviceInfo', 'mobileDeviceMarketingName', 'flashVersion', 
                  'language', 'screenColors', 'screenResolution','adwordsClickInfo.isVideoAd',
                  'cityId', 'latitude', 'longitude', 'networkLocation', 'adwordsClickInfo.slot',
                  'adwordsClickInfo.gclId','adwordsClickInfo.page',
                  'device', 'geoNetwork', 'totals', 'trafficSource', 'customDimensions', 'visits', 'hits_x',
                  'adwordsClickInfo.adNetworkType', 'adContent',
                  'totalTransactionRevenue', 'visitId'], axis=1)


In [74]:
df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85417 entries, 0 to 85416
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   channelGrouping       85417 non-null  object
 1   date                  85417 non-null  int64 
 2   fullVisitorId         85417 non-null  object
 3   socialEngagementType  85417 non-null  object
 4   visitNumber           85417 non-null  int64 
 5   visitStartTime        85417 non-null  int64 
 6   browser               85417 non-null  object
 7   operatingSystem       85417 non-null  object
 8   isMobile              85417 non-null  bool  
 9   deviceCategory        85417 non-null  object
 10  continent             85417 non-null  object
 11  subContinent          85417 non-null  object
 12  country               85417 non-null  object
 13  region                85417 non-null  object
 14  metro                 85417 non-null  object
 15  city                  85417 non-null

In [75]:
#Set dtype
df_2['pageviews'] = df_2['pageviews'].astype(int)
df_2['newVisits'] = df_2['newVisits'].astype(int)
df_2['bounces'] = df_2['bounces'].astype(int)
df_2['sessionQualityDim'] = df_2['sessionQualityDim'].astype(int)
df_2['timeOnSite'] = df_2['timeOnSite'].astype(int)
df_2['transactions'] = df_2['transactions'].astype(int)
df_2['transactionRevenue'] = df_2['transactionRevenue'].astype(int)
df_2['fullVisitorId'] = df_2['fullVisitorId'].astype(str)

In [76]:
df_2['hits'] = df_2['hits_y']

In [77]:
df_2 = df_2.drop(['hits_y'], axis = 1)

In [78]:
#Create Day, month, year columns
df['date'] = pd.to_datetime(df['visitStartTime'], unit='s')
df_2['weekday'] = df_2['date'].dt.weekday
df_2['day'] = df_2['date'].dt.day
df_2['month'] = df_2['date'].dt.month
df_2['year'] = df_2['date'].dt.year

NameError: name 'df' is not defined

In [48]:
df_2 = df_2.drop(['date'], axis = 1)

In [49]:
#Make isMobile an int
df_2['isMobile'] = df_2['isMobile'].replace(True, 1)
df_2['isMobile'] = df_2['isMobile'].replace(False, 0)
df_2['isTrueDirect'] = df_2['isTrueDirect'].replace(False, 0)

In [50]:
#Create an other in browser
mask = df_2.browser.map(df_2.browser.value_counts()) < 200
df_2.browser =  df_2.browser.mask(mask, 'Other')
df_2['browser'].replace([np.nan], 'Other', inplace = True)
df_2['browser'].value_counts()

Chrome               58299
Safari               15752
Firefox               3271
Internet Explorer     1816
Android Webview       1746
Edge                  1028
Safari (in-app)        765
Samsung Internet       762
Opera Mini             737
Opera                  507
Other                  462
UC Browser             272
Name: browser, dtype: int64

In [51]:
mask = df_2.country.map(df_2.country.value_counts()) < 20
df_2.country =  df_2.country.mask(mask, 'Other')
df_2['country'].replace([np.nan, '(not set)'], 'Other', inplace = True)
df_2['country'].value_counts()

United States           35643
India                    5246
United Kingdom           3640
Canada                   2596
Germany                  1881
Japan                    1813
Brazil                   1752
Vietnam                  1705
France                   1617
Thailand                 1547
Turkey                   1498
Taiwan                   1409
Mexico                   1296
Spain                    1208
Australia                1196
Netherlands              1132
Italy                    1028
Russia                   1009
Indonesia                 814
Poland                    784
Philippines               707
Singapore                 681
South Korea               653
Other                     634
Ireland                   632
Malaysia                  536
Argentina                 500
Romania                   498
Ukraine                   490
Sweden                    489
Israel                    475
Peru                      474
Colombia                  446
Belgium   

In [52]:
mask = df_2.metro.map(df_2.metro.value_counts()) < 10
df_2.metro =  df_2.metro.mask(mask, 'Other')
df_2['metro'].replace([np.nan, '(not set)', 'not available in demo dataset'], 'Other', inplace = True)
df_2['metro'].value_counts()

Other                                     66363
San Francisco-Oakland-San Jose CA          9040
New York NY                                2480
London                                     1189
Los Angeles CA                              969
Chicago IL                                  771
Seattle-Tacoma WA                           769
Washington DC (Hagerstown MD)               367
Austin TX                                   353
Boston MA-Manchester NH                     303
Dallas-Ft. Worth TX                         289
Atlanta GA                                  286
Houston TX                                  266
Detroit MI                                  219
JP_KANTO                                    207
San Diego CA                                159
Roanoke-Lynchburg VA                        150
Philadelphia PA                             144
Denver CO                                   119
Pittsburgh PA                                98
Portland OR                             

In [53]:
mask = df_2.campaign.map(df_2.campaign.value_counts()) < 10
df_2.campaign =  df_2.campaign.mask(mask, 'Other')
df_2['campaign'].replace([np.nan, '(not set)'], 'Other', inplace = True)
df_2['campaign'].value_counts()

Other                                                                   80278
Data Share Promo                                                         1682
1000557 | GA | US | en | Hybrid | GDN Text+Banner | AS                   1227
1000557 | GA | US | en | Hybrid | GDN Remarketing                         724
AW - Dynamic Search Ads Whole Site                                        713
AW - Accessories                                                          413
Smart Display Campaign                                                    142
"google + redesign/Accessories March 17" All Users Similar Audiences       53
Page: contains "/google+redesign/drinkware" Similar Audiences              27
"google + redesign/Accessories March 17" All Users                         25
Sports & Fitness/Health & Fitness Buffs                                    24
Media & Entertainment/Music Lovers                                         16
Run of Network Line Item                                        

In [54]:
mask = df_2.city.map(df_2.city.value_counts()) < 5
df_2.city =  df_2.city.mask(mask, 'Other')
df_2['city'].replace([np.nan, '(not set)', 'not available in demo dataset'], 'Other', inplace = True)
df_2['city'].value_counts()

Other                         50703
Mountain View                  3593
New York                       2442
San Francisco                  1868
Sunnyvale                      1433
London                         1181
San Jose                        995
Los Angeles                     819
Chicago                         764
Bangkok                         641
Bengaluru                       532
Istanbul                        523
Ho Chi Minh City                511
Toronto                         496
Seattle                         482
Dublin                          479
Mumbai                          452
Paris                           423
Sydney                          419
Chennai                         417
Hyderabad                       403
Sao Paulo                       372
Austin                          353
Singapore                       344
Hanoi                           336
Santa Clara                     330
Jakarta                         323
Seoul                       

In [55]:
mask = df_2.region.map(df_2.region.value_counts()) < 10
df_2.region =  df_2.region.mask(mask, 'Other')
df_2['region'].replace([np.nan, '(not set)', 'not available in demo dataset'], 'Other', inplace = True)
df_2['region'].value_counts()

Other                                49763
California                           10185
New York                              2461
England                               1286
Texas                                  992
Illinois                               778
Washington                             777
Maharashtra                            696
Bangkok                                641
Taipei City                            634
Ontario                                565
Delhi                                  547
Karnataka                              532
Istanbul                               523
Tamil Nadu                             487
Ile-de-France                          459
New South Wales                        420
County Dublin                          417
Telangana                              404
State of Sao Paulo                     387
Ho Chi Minh                            353
Hanoi                                  345
Tokyo                                  343
Jakarta    

In [56]:
mask = df_2.networkDomain.map(df_2.networkDomain.value_counts()) < 2
df_2.networkDomain =  df_2.networkDomain.mask(mask, 'Other')
df_2['networkDomain'].replace([np.nan, '(not set)', 'unknown.unknown'], 'Other', inplace = True)
df_2['networkDomain'].value_counts()

Other                          43066
comcast.net                     2776
rr.com                          1378
verizon.net                     1318
ttnet.com.tr                     916
hinet.net                        861
comcastbusiness.net              848
virginm.net                      603
sbcglobal.net                    540
prod-infinitum.com.mx            529
cox.net                          518
google.com                       497
optonline.net                    490
btcentralplus.com                466
att.net                          459
3bb.co.th                        424
rima-tde.net                     387
vnpt.vn                          382
totbb.net                        353
asianet.co.th                    344
qwest.net                        330
t-ipconnect.de                   314
bell.ca                          305
pldt.net                         298
telecomitalia.it                 297
virtua.com.br                    287
shawcable.net                    280
a

In [57]:
#Create an other in operatingSystem
mask = df_2.operatingSystem.map(df_2.operatingSystem.value_counts()) < 200
df_2.operatingSystem =  df_2.operatingSystem.mask(mask, 'Other')
df_2['operatingSystem'].replace([np.nan, '(not set)'], 'Other', inplace = True)
df_2['operatingSystem'].value_counts()

Windows      31012
Macintosh    21920
Android      14837
iOS          11124
Linux         3192
Chrome OS     2558
Other          774
Name: operatingSystem, dtype: int64

In [58]:
#Create an other in source
mask = df_2.source.map(df_2.source.value_counts()) < 200
df_2.source =  df_2.source.mask(mask, 'Other')
df_2['source'].replace([np.nan], 'Other', inplace = True)
df_2['source'].value_counts()

google                         32811
(direct)                       28233
youtube.com                    16602
Other                           2039
analytics.google.com            1778
Partners                        1682
sites.google.com                 466
google.com                       391
m.facebook.com                   380
dfa                              346
baidu                            266
reddit.com                       217
googleads.g.doubleclick.net      206
Name: source, dtype: int64

In [59]:
#Create an other in subContinent
mask = df_2.subContinent.map(df_2.subContinent.value_counts()) < 200
df_2.subContinent =  df_2.subContinent.mask(mask, 'Other')
df_2['subContinent'].replace([np.nan], 'Other', inplace = True)
df_2['subContinent'].value_counts()

Northern America    38243
Southern Asia        6090
Southeast Asia       6067
Western Europe       5767
Northern Europe      5552
Eastern Asia         4657
South America        3694
Eastern Europe       3687
Southern Europe      3412
Western Asia         3117
Central America      1483
Australasia          1421
Northern Africa       875
Western Africa        336
Southern Africa       296
Other                 289
Caribbean             219
Eastern Africa        212
Name: subContinent, dtype: int64

In [60]:
mask = df_2.keyword.map(df_2.keyword.value_counts()) < 2
df_2.keyword =  df_2.keyword.mask(mask, 'Other')
df_2['keyword'].replace([np.nan, '(not provided)', '0'], 'Other', inplace = True)
df_2['keyword'].value_counts()

Other                                                                                  81528
(User vertical targeting)                                                               1318
(automatic matching)                                                                     924
6qEhsCssdK0z36ri                                                                         545
(Remarketing/Content targeting)                                                          264
google merchandise store                                                                 136
1hZbAqLCbjwfgOH7                                                                         112
Google Merchandise                                                                       101
1X4Me6ZKNV0zg-jV                                                                          55
google store                                                                              51
youtube                                                               

In [61]:
mask = df_2.referralPath.map(df_2.referralPath.value_counts()) < 2
df_2.referralPath =  df_2.referralPath.mask(mask, 'Other')
df_2['referralPath'].replace([np.nan, '0'], 'Other', inplace = True)
df_2['referralPath'].value_counts()

Other                                                                                                            57524
/                                                                                                                 6807
/yt/about/                                                                                                        3857
/analytics/web/                                                                                                   1552
/yt/about/tr/                                                                                                      735
/yt/about/vi/                                                                                                      687
/yt/about/pt-BR/                                                                                                   625
/yt/about/es-419/                                                                                                  624
/yt/about/th/                                   

In [62]:
df_2['medium'].replace([np.nan, '(none)', '(not set)'], 'Other', inplace = True)

In [63]:
df_2['fullVisitorId'].duplicated().sum()

2691

In [93]:
df_trans = df_2.groupby(['fullVisitorId']).agg({'transactions': 'sum'}).reset_index()

In [94]:
df_trans.columns = ['fullVisitorId', 'total_transactions']

In [95]:
df_2 = df_2.merge(df_trans, how='left', on=['fullVisitorId'])

In [97]:
df_2['return_customer'] = np.nan
df_2['return_customer'] = np.where(df_2['transactions'] == 0, 0, df_2['return_customer'])
df_2['return_customer'] = np.where(df_2['transactions'] == 1, 0, df_2['return_customer'])
df_2['return_customer'] = np.where(df_2['transactions'] > 1, 1, df_2['return_customer'])

In [98]:
df_2.to_csv('df_full.csv', index=False)

In [64]:
#Group the customers by fullVisitorId
df_3 = df_2.groupby(['fullVisitorId']).agg({'channelGrouping': lambda x:x.value_counts(dropna=False).index[0], 
                                        'socialEngagementType': 'max',
                                         'visitNumber': 'max', 
                                         'visitStartTime': 'mean',
                                         'browser': lambda x:x.value_counts(dropna=False).index[0],
                                         'operatingSystem': lambda x:x.value_counts(dropna=False).index[0],
                                         'isMobile': lambda x:x.value_counts(dropna=False).index[0],
                                         'deviceCategory': lambda x:x.value_counts(dropna=False).index[0],
                                         'continent': lambda x:x.value_counts(dropna=False).index[0],
                                         'subContinent': lambda x:x.value_counts(dropna=False).index[0],
                                         'country': lambda x:x.value_counts(dropna=False).index[0],
                                         'region': lambda x:x.value_counts(dropna=False).index[0],
                                         'metro': lambda x:x.value_counts(dropna=False).index[0],
                                         'city': lambda x:x.value_counts(dropna=False).index[0],
                                         'networkDomain': lambda x:x.value_counts(dropna=False).index[0],
                                         'hits': 'sum',
                                         'pageviews': 'sum',
                                         'bounces': 'sum',
                                         'newVisits': 'sum',
                                         'sessionQualityDim': 'mean',
                                         'timeOnSite': 'sum',
                                         'transactions': 'sum',
                                         'transactionRevenue': 'sum',
                                         'source': lambda x:x.value_counts(dropna=False).index[0],
                                         'campaign': lambda x:x.value_counts(dropna=False).index[0],
                                         'medium': lambda x:x.value_counts(dropna=False).index[0],
                                         'keyword': lambda x:x.value_counts(dropna=False).index[0],
                                         'referralPath': lambda x:x.value_counts(dropna=False).index[0],
                                         'isTrueDirect': lambda x:x.value_counts(dropna=False).index[0],
                                         'campaignCode': lambda x:x.value_counts(dropna=False).index[0],
                                         'weekday': 'max',
                                         'day': 'max',
                                         'month': 'max',
                                         'year': 'max'
                                        }).reset_index()

In [80]:
df_3['return_customer'] = np.nan
df_3['return_customer'] = np.where(df_3['transactions'] == 0, 0, df_3['return_customer'])
df_3['return_customer'] = np.where(df_3['transactions'] == 1, 0, df_3['return_customer'])
df_3['return_customer'] = np.where(df_3['transactions'] > 1, 1, df_3['return_customer'])

In [81]:
df_3.to_csv('df_combined.csv', index=False)